# Masuyama 氏らの提案手法に基づく位相復元 (ADMM法とする)
Y. Masuyama, K. Yatabe and Y. Oikawa, "Griffin-Lim like phase recovery via alternating directionmethod of multipliers," IEEE Signal Processing Letters, vol.26, no.1, pp.184--188, Jan. 2019. https://ieeexplore.ieee.org/document/8552369

In [1]:
import numpy as np
from scipy.io import wavfile
import librosa
from pypesq import pesq
from IPython.display import Audio

In [2]:
IN_WAVE_FILE = "in.wav"  # モノラル音声
OUT_WAVE_FILE = "out_admm_gla.wav"  # 復元音声

In [3]:
FRAME_LENGTH = 1024        # フレーム長 (FFTサイズ)
HOP_LENGTH = 80            # フレームのシフト長
ITERATION = 100            # 位相推定の最大繰り返し数
MULTIPLIER = 0.01          # ADMM法の強さを制御; 0.0のときはGriffin-Lim法に一致

In [4]:
# 音声のロード
fs, data = wavfile.read(IN_WAVE_FILE)
data = data.astype(np.float64)

In [5]:
# 振幅スペクトル（位相復元なので手に入るのはこれのみ）
amp_spec = np.abs(
    librosa.core.stft(
        data, n_fft=FRAME_LENGTH, hop_length=HOP_LENGTH, win_length=FRAME_LENGTH
    )
)

In [6]:
# 乱数の種を指定して再現性を保証
np.random.seed(seed=0)

In [7]:
# ADMM法に基づく位相スペクトルの推定
for i in range(ITERATION):
    if i == 0:
        # 初回は乱数で初期化
        phase_spec = np.random.rand(*amp_spec.shape)
        control_spec = np.zeros(amp_spec.shape)
    else:
        # 振幅スペクトルと推定された位相スペクトルから複素スペクトログラムを復元
        recovered_spec = amp_spec * np.exp(1j * phase_spec)

        # 短時間フーリエ逆変換で音声を復元
        combined = recovered_spec + control_spec
        recovered = librosa.core.istft(
            combined, hop_length=HOP_LENGTH, win_length=FRAME_LENGTH
        )

        # 復元音声から複素スペクトログラムを再計算
        complex_spec = librosa.core.stft(
            recovered,
            n_fft=FRAME_LENGTH,
            hop_length=HOP_LENGTH,
            win_length=FRAME_LENGTH,
        )
        complex_spec = MULTIPLIER * combined + complex_spec
        complex_spec /= 1.0 + MULTIPLIER

        # 初回以降は計算済みの複素スペクトログラムから位相スペクトルを推定
        control_spec = control_spec + recovered_spec - complex_spec
        phase_spec = np.angle(complex_spec - control_spec)

In [8]:
# 音声を復元
recovered_spec = amp_spec * np.exp(1j * phase_spec)
recovered = librosa.core.istft(
    recovered_spec, hop_length=HOP_LENGTH, win_length=FRAME_LENGTH
)
# pesqで音質を評価
print(pesq(recovered, data[: len(recovered)], fs))
Audio(recovered, rate=fs) # 再生

4.314631462097168


In [9]:
# 比較のため、Griffin-Lim法により位相復元
recovered = librosa.griffinlim(
    amp_spec, n_iter=ITERATION, hop_length=HOP_LENGTH, random_state=0
)

# pesqで音質を評価
print(pesq(recovered, data[: len(recovered)], fs))
Audio(recovered, rate=fs) # 再生

4.310971736907959
